In [ ]:
%matplotlib inline
from __future__ import print_function
import os
import sys
import copy
import numpy as np
import tensorflow as tf
import lxml.etree as et

import convnet
import mutate
import convevo

In [ ]:
reload (convnet)
reload (mutate)
reload (convevo)

In [ ]:
# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [ ]:
from six.moves import cPickle as pickle
pickle_file = '../ud730/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
datasets = {
    "image_size": 28,
    "label_count": 10,
    "channel_count": 1
}
datasets["total_image_size"] = datasets["image_size"] * datasets["image_size"]

def reformat(dataset, labels, name):
    dataset = dataset.reshape((-1, datasets["image_size"], datasets["image_size"], datasets["channel_count"])).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(datasets["label_count"]) == labels[:,None]).astype(np.float32)
    print(name + " set", dataset.shape, labels.shape)
    return dataset, labels
datasets["train"], datasets["train_labels"] = reformat(train_dataset, train_labels, "Training")
datasets["valid"], datasets["valid_labels"] = reformat(valid_dataset, valid_labels, "Validation")
datasets["test"], datasets["test_labels"] = reformat(test_dataset, test_labels, "Test")

print(datasets.keys())

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [ ]:
def run_graph(graph_info, data, step_count, report_every=50):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        batch_size = graph_info["batch_size"]
        for step in xrange(step_count + 1):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (data["train_labels"].shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = data["train"][offset:(offset + batch_size), :, :, :]
            batch_labels = data["train_labels"][offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            targets = [graph_info["optimizer"], graph_info["loss"], graph_info["predictions"]]
            feed_dict = {graph_info["train"] : batch_data, graph_info["labels"] : batch_labels}
            _, l, predictions = session.run(targets, feed_dict=feed_dict)
            if np.isnan(l):
                print("Error computing loss")
                return 0
            if (step % report_every == 0):
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(graph_info["valid"].eval(), data["valid_labels"]))
        test_accuracy = accuracy(graph_info["test"].eval(), data["test_labels"])
        print("Test accuracy: %.1f%%" % test_accuracy)
        return test_accuracy

In [ ]:
def shape_test(shape, options, func):
    graph = tf.Graph()
    with graph.as_default():
        input = tf.placeholder(tf.float32, shape=shape)
        parameters = convnet.setup_matrix(options)
        result = func(input, False, parameters, options)
        return tuple(int(d) for d in result.get_shape())
    
default_init = convnet.setup_initializer()
correct = 0
for w in xrange(1, 7):
    for p in xrange(1, w + 1):
        for s in xrange(1, p + 1):
            for pad in ["SAME", "VALID"]:
                for func in [convnet.apply_pool, convnet.apply_conv]:
                    options = {
                        "size":(p, p, 1, 1),
                        "stride": (s, s),
                        "padding":pad,
                        "pool_type": "max",
                        "bias":False,
                        "init":default_init}
                    calc = convnet.image_output_shape([1, w, w, 1], options)
                    shape = shape_test([1, w, w, 1], options, func)
                    if calc == shape:
                        correct += 1
                    else:
                        print("mismatch for", w, p, s, pad, shape, calc)
correct

In [ ]:
def convnet_two_layer(batch_size, patch_size, depth, hidden_size, data):
    image_size = data["image_size"]
    label_count = data["label_count"]
    channel_count = data["channel_count"]
    graph = tf.Graph()
    with graph.as_default():
        # Input data.
        input_shape = (batch_size, image_size, image_size, channel_count)
        output_shape = (batch_size, label_count)
        train = tf.placeholder(tf.float32, shape=input_shape)
        labels= tf.placeholder(tf.float32, shape=output_shape)
        valid = tf.constant(data["valid"])
        test  = tf.constant(data["test"])
        
        stride = 2
        
        layers = [
            convnet.create_conv_layer((patch_size, patch_size), (stride, stride), 1, depth),
            convnet.create_relu_layer(),
            convnet.create_conv_layer((patch_size, patch_size), (stride, stride), depth, depth),
            convnet.create_relu_layer(),
            convnet.create_flatten_layer(),
            convnet.create_matrix_layer(image_size * image_size * depth / pow(stride, 4), hidden_size),
            convnet.create_relu_layer(),
            convnet.create_matrix_layer(hidden_size, label_count)
        ]
        
        for layer in layers:
            layer.setup_parameters()
        
        def model(nodes, train):
            for layer in layers:
                nodes.append(layer.connect(nodes[-1], train))
            return nodes[-1]

        logits = model([train], True)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "labels": labels,
            "loss": loss,
            "optimizer": tf.train.GradientDescentOptimizer(0.05).minimize(loss),

            # Predictions for the training, validation, and test data.
            "predictions": tf.nn.softmax(logits),
            "valid": tf.nn.softmax(model([valid], False)),
            "test":  tf.nn.softmax(model([test], False))
        }
    return info

In [ ]:
graph_2conv = convnet_two_layer(batch_size=16, patch_size=5, depth=16, hidden_size=64, data=datasets)

run_graph(graph_2conv, datasets, 1000)

In [ ]:
def convnet_two_layer_stack(batch_size, patch_size, depth, hidden_size, data):
    image_size = data["image_size"]
    label_count = data["label_count"]
    channel_count = data["channel_count"]
    graph = tf.Graph()
    with graph.as_default():
        # Input data.
        input_shape = (batch_size, image_size, image_size, channel_count)
        output_shape = (batch_size, label_count)
        train = tf.placeholder(tf.float32, shape=input_shape)
        labels= tf.placeholder(tf.float32, shape=output_shape)
        valid = tf.constant(data["valid"])
        test  = tf.constant(data["test"])
        
        stride = 2
        
        evo_stack = convevo.LayerStack(flatten=True)
        default_init = lambda: convevo.Initializer("normal", scale=1)
        
        evo_stack.add_layer(convevo.ImageLayer("conv_bias", patch_size, stride, depth, "SAME", default_init()), relu=True)
        evo_stack.add_layer(convevo.ImageLayer("conv_bias", patch_size, stride, depth, "SAME", default_init()), relu=True)
        evo_stack.add_layer(convevo.HiddenLayer(hidden_size, bias=True, initializer=default_init()), relu=True)
        evo_stack.add_layer(convevo.HiddenLayer(label_count, bias=True, initializer=default_init()), relu=False)
        
        #print(et.tostring(evo_stack.to_xml(), pretty_print=True))
        
        evo_copy = copy.deepcopy(evo_stack)
        evo_copy.mutate(55)
        
        layers = evo_copy.construct(input_shape)
        
        for layer in layers:
            layer.setup_parameters()
        
        def model(nodes, train):
            for layer in layers:
                nodes.append(layer.connect(nodes[-1], train))
            return nodes[-1]

        logits = model([train], True)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "labels": labels,
            "loss": loss,
            "optimizer": tf.train.GradientDescentOptimizer(0.05).minimize(loss),

            # Predictions for the training, validation, and test data.
            "predictions": tf.nn.softmax(logits),
            "valid": tf.nn.softmax(model([valid], False)),
            "test":  tf.nn.softmax(model([test], False))
        }
    return info

In [ ]:
tf.set_random_seed(42)

graph_2conv_stack = convnet_two_layer_stack(batch_size=16, patch_size=5, depth=16, hidden_size=64, data=datasets)

run_graph(graph_2conv, datasets, 1000)

In [ ]:
optimal_steps = 200000

graph_connive = convnet_optimize(
    batch_size=16, patch_sizes=[5,14], strides=[2,7], depths=[16,128],
    hidden_sizes=[128,64],
    rate_alpha=0.02, decay_rate=0.9, decay_steps=optimal_steps/4,
    beta_loss=0.0005,
    dropout_rate=0.5,
    base_seed=45645,
    data=datasets)

run_graph(graph_connive, datasets, optimal_steps, report_every=1000)

In [ ]:
reload(darwin)

evo = darwin.Darwin(range(0, 10), lambda i: str(i), lambda i, e: i, lambda i, e: i + e.randint(1,20))
results = evo.evaluate(random.Random(42))
evo.repopulate(0.1, results, random.Random(42))
evo.evaluate(random.Random(42))
print(evo.best())